In [1]:
!pip install peft
!pip install bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 866.1 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.7 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
import pickle
import torchaudio
import accelerate
from tqdm import tqdm
from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader
from peft import prepare_model_for_kbit_training, PeftModel
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor, BitsAndBytesConfig
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language='en', task = "transcribe")
whisper_tokenizer = WhisperTokenizer.from_pretrained('openai/whisper-large-v3', language = 'en', task = "transcribe")
test_dataset = torchaudio.datasets.LIBRISPEECH('/kaggle/input/librispeech', url="test-clean", download=False)

2024-08-17 00:17:54.235313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 00:17:54.235412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 00:17:54.352851: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = WhisperForConditionalGeneration.from_pretrained('openai/whisper-large-v3', quantization_config = bnb_config, device_map=1)
custom_model = PeftModel.from_pretrained(model, '/kaggle/input/audio-recognition/checkpoint-1500/', download=False)
q_model = prepare_model_for_kbit_training(custom_model, use_gradient_checkpointing=False)
q_model.config.use_cache = True
q_model = torch.compile(q_model, mode='max-autotune')

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

In [ ]:
def total_params(model):
    return sum(p.numel() for p in model.parameters())

print(f'Memory used by model: {round(q_model.get_memory_footprint()/1024/1024/1024, 2)} GB')
print(total_params(q_model))

In [ ]:
class training_dataset(Dataset) :
    def __init__(self, dataset) :
        super().__init__()
        self.data = dataset
        self.labels = []
    def __len__(self) :
        return self.data.__len__()
    
    def get_labels(self) :
        return self.labels
    
    def __getitem__(self, idx) :
        output = {}
        data = processor(self.data[idx][0].numpy(), sampling_rate = 16000, truncation=True, padding_size=3000, return_tensors='pt', return_attention_mask=True)
        
        output['data'] = data['input_features'].squeeze(0)
        output['attention_mask'] = data['attention_mask']
        self.labels.append(whisper_tokenizer(self.data[idx][2], padding='longest', truncation=True, return_tensors='pt').input_ids)
        
        return output
    
dataset = training_dataset(test_dataset)
loader = DataLoader(dataset, 16)

In [ ]:
outputs = []
labels = []

q_model.eval()
for batch in tqdm(loader, total = 328) :
    with torch.cuda.amp.autocast() :
            output = q_model.generate(input_features = batch['data'], attention_mask = batch['attention_mask'])
            outputs.extend(output)

In [ ]:
with open('/kaggle/working/sequences.pkl', 'wb') as f:
    pickle.dump(outputs, f)
    
with open('/kaggle/working/labels.pkl', 'wb') as f:
    pickle.dump(dataset.get_labels(), f)